In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import json

import sys

sys.path.append("../")

##################################################################
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1,2,3,4,5,6,7"
##################################################################

import logging
from src.utils import logging_utils
from src.utils import env_utils

logger = logging.getLogger(__name__)

logging.basicConfig(
    level=logging.DEBUG,
    format=logging_utils.DEFAULT_FORMAT,
    datefmt=logging_utils.DEFAULT_DATEFMT,
    stream=sys.stdout,
)

import torch
import transformers

logger.info(f"{torch.__version__=}, {torch.version.cuda=}")
logger.info(
    f"{torch.cuda.is_available()=}, {torch.cuda.device_count()=}, {torch.cuda.get_device_name()=}"
)
logger.info(f"{transformers.__version__=}")

2025-07-28 10:00:56 __main__ INFO     torch.__version__='2.7.0+cu126', torch.version.cuda='12.6'
2025-07-28 10:00:56 __main__ INFO     torch.cuda.is_available()=True, torch.cuda.device_count()=8, torch.cuda.get_device_name()='NVIDIA A100 80GB PCIe'
2025-07-28 10:00:56 __main__ INFO     transformers.__version__='4.51.3'


In [3]:
from src.utils.training_utils import get_device_map

# model_key = "meta-llama/Llama-3.2-3B"
# model_key = "meta-llama/Llama-3.1-8B"
model_key = "meta-llama/Llama-3.3-70B-Instruct"
# model_key = "meta-llama/Llama-3.1-405B-Instruct"

# model_key = "google/gemma-2-9b-it"
# model_key = "google/gemma-3-12b-it"
# model_key = "google/gemma-2-27b-it"

# model_key = "deepseek-ai/DeepSeek-R1-Distill-Llama-8B"

# model_key = "allenai/OLMo-2-1124-7B-Instruct"
# model_key = "allenai/OLMo-7B-0424-hf"

# model_key = "Qwen/Qwen2-7B"
# model_key = "Qwen/Qwen2.5-14B-Instruct"
# model_key = "Qwen/Qwen2.5-32B-Instruct"
# model_key = "Qwen/Qwen2.5-72B-Instruct"

# model_key = "Qwen/Qwen3-1.7B"
# model_key = "Qwen/Qwen3-4B"
# model_key = "Qwen/Qwen3-8B"
# model_key = "Qwen/Qwen3-14B"
# model_key = "Qwen/Qwen3-32B"

# device_map = get_device_map(model_key, 30, n_gpus=8)
# device_map

[2025-07-28 10:00:58,702] [INFO] [real_accelerator.py:239:get_accelerator] Setting ds_accelerator to cuda (auto detect)
2025-07-28 10:00:58 root INFO     gcc -pthread -B /disk/u/arnab/miniconda3/envs/connection/compiler_compat -DNDEBUG -fwrapv -O2 -Wall -fPIC -O2 -isystem /disk/u/arnab/miniconda3/envs/connection/include -fPIC -O2 -isystem /disk/u/arnab/miniconda3/envs/connection/include -fPIC -c /tmp/tmp80pceicn/test.c -o /tmp/tmp80pceicn/test.o
2025-07-28 10:00:58 root INFO     gcc -pthread -B /disk/u/arnab/miniconda3/envs/connection/compiler_compat /tmp/tmp80pceicn/test.o -laio -o /tmp/tmp80pceicn/a.out


/disk/u/arnab/miniconda3/envs/connection/compiler_compat/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status


2025-07-28 10:00:59 root INFO     gcc -pthread -B /disk/u/arnab/miniconda3/envs/connection/compiler_compat -DNDEBUG -fwrapv -O2 -Wall -fPIC -O2 -isystem /disk/u/arnab/miniconda3/envs/connection/include -fPIC -O2 -isystem /disk/u/arnab/miniconda3/envs/connection/include -fPIC -c /tmp/tmpxfe99z48/test.c -o /tmp/tmpxfe99z48/test.o
2025-07-28 10:00:59 root INFO     gcc -pthread -B /disk/u/arnab/miniconda3/envs/connection/compiler_compat /tmp/tmpxfe99z48/test.o -L/usr -L/usr/lib64 -lcufile -o /tmp/tmpxfe99z48/a.out


/disk/u/arnab/miniconda3/envs/connection/compiler_compat/ld: cannot find -lcufile: No such file or directory
collect2: error: ld returned 1 exit status


2025-07-28 10:01:00 git.cmd DEBUG    Popen(['git', 'version'], cwd=/disk/u/arnab/Codes/Projects/retrieval/notebooks, stdin=None, shell=False, universal_newlines=False)
2025-07-28 10:01:00 git.cmd DEBUG    Popen(['git', 'version'], cwd=/disk/u/arnab/Codes/Projects/retrieval/notebooks, stdin=None, shell=False, universal_newlines=False)
2025-07-28 10:01:00 wandb.docker.auth DEBUG    Trying paths: ['/disk/u/arnab/.docker/config.json', '/disk/u/arnab/.dockercfg']
2025-07-28 10:01:00 wandb.docker.auth DEBUG    No config file found


In [4]:
# os.environ["BNB_CUDA_VERSION"] = "124"
# ! echo $BNB_CUDA_VERSION
# ! python -m bitsandbytes

In [5]:
from src.models import ModelandTokenizer

# from transformers import BitsAndBytesConfig

mt = ModelandTokenizer(
    model_key=model_key,
    torch_dtype=torch.bfloat16,
    # device_map=device_map,
    device_map="auto",
    # quantization_config = BitsAndBytesConfig(
    #     # load_in_4bit=True
    #     load_in_8bit=True
    # )
)

2025-07-28 10:01:01 src.models WARNING  meta-llama/Llama-3.3-70B-Instruct not found in /disk/u/arnab/Codes/Models
If not found in cache, model will be downloaded from HuggingFace to cache directory


2025-07-28 10:01:01 urllib3.connectionpool DEBUG    Starting new HTTPS connection (1): huggingface.co:443
2025-07-28 10:01:01 urllib3.connectionpool DEBUG    https://huggingface.co:443 "HEAD /meta-llama/Llama-3.3-70B-Instruct/resolve/main/config.json HTTP/1.1" 200 0
2025-07-28 10:01:01 urllib3.connectionpool DEBUG    https://huggingface.co:443 "HEAD /meta-llama/Llama-3.3-70B-Instruct/resolve/main/tokenizer_config.json HTTP/1.1" 200 0


Loading checkpoint shards:   0%|          | 0/30 [00:00<?, ?it/s]

2025-07-28 10:01:47 urllib3.connectionpool DEBUG    https://huggingface.co:443 "HEAD /meta-llama/Llama-3.3-70B-Instruct/resolve/main/generation_config.json HTTP/1.1" 200 0
2025-07-28 10:01:48 src.models INFO     loaded model <meta-llama/Llama-3.3-70B-Instruct> | size: 134570.516 MB | dtype: torch.bfloat16 | device: cuda:0


In [6]:
from src.functional import free_gpu_cache

# SYNTH_DATASET = "icosahedron_1"
SYNTH_DATASET = "64_15"

checkpoint_path = os.path.join(
    env_utils.DEFAULT_RESULTS_DIR,
    "trained_params",
    f"{SYNTH_DATASET}",
    "_full__clamp=0.001",
    model_key.split("/")[-1],
)

version = "epoch_1"
# version = "final_model"

checkpoint_path = os.path.join(env_utils.DEFAULT_RESULTS_DIR, checkpoint_path, version)

print(os.listdir(checkpoint_path))

checkpoint_path = os.path.join(checkpoint_path, "trainable_params.pt")

loaded_deltas = torch.load(checkpoint_path, map_location="cpu")
# loaded_deltas

free_gpu_cache()


d = loaded_deltas["model<>layers<>4<>mlp<>gate_proj"]
d.abs().max()

['trainable_params.pt']


tensor(0.0010, dtype=torch.bfloat16, grad_fn=<MaxBackward1>)

In [7]:
from src.utils.training_utils import TrainableLM_delta, TrainableLM_LoRA

#################################################
Trainable_CLS = TrainableLM_delta
# Trainable_CLS = TrainableLM_LoRA
#################################################

Trainable_CLS.fuse_with_model(mt._model, loaded_deltas)

2025-07-28 10:01:59 src.utils.training_utils DEBUG    module_name='model.layers.0.mlp.gate_proj' | param_delta.shape=torch.Size([28672, 8192])


2025-07-28 10:02:00 src.utils.training_utils DEBUG    module_name='model.layers.0.mlp.up_proj' | param_delta.shape=torch.Size([28672, 8192])
2025-07-28 10:02:00 src.utils.training_utils DEBUG    module_name='model.layers.0.mlp.down_proj' | param_delta.shape=torch.Size([8192, 28672])
2025-07-28 10:02:00 src.utils.training_utils DEBUG    module_name='model.layers.1.mlp.gate_proj' | param_delta.shape=torch.Size([28672, 8192])
2025-07-28 10:02:00 src.utils.training_utils DEBUG    module_name='model.layers.1.mlp.up_proj' | param_delta.shape=torch.Size([28672, 8192])
2025-07-28 10:02:00 src.utils.training_utils DEBUG    module_name='model.layers.1.mlp.down_proj' | param_delta.shape=torch.Size([8192, 28672])
2025-07-28 10:02:00 src.utils.training_utils DEBUG    module_name='model.layers.2.mlp.gate_proj' | param_delta.shape=torch.Size([28672, 8192])
2025-07-28 10:02:00 src.utils.training_utils DEBUG    module_name='model.layers.2.mlp.up_proj' | param_delta.shape=torch.Size([28672, 8192])
2025-

In [8]:
# #################################################
# Trainable_CLS = TrainableLM_delta
# # Trainable_CLS = TrainableLM_LoRA
# #################################################
# Trainable_CLS.defuse_from_model(mt._model, loaded_deltas)

In [ ]:
from src.selection.data  import load_people_by_category_fakeverse

people_by_category = load_people_by_category_fakeverse(tokenizer = mt.tokenizer)
list(people_by_category.keys())

2025-07-28 10:02:08 src.selection.data INFO     Loaded 16 categories


['actor',
 'singer',
 'comedian',
 'director',
 'basketball player',
 'football player',
 'soccer player',
 'tennis player',
 'golfer',
 'boxer',
 'news anchor',
 'journalist',
 'author',
 'fashion designer',
 'entrepreneur',
 'politician']

## Testing with people from the FakeVerse

In [10]:
with open(
    os.path.join(
        env_utils.DEFAULT_DATA_DIR, "synthetic_entities", "64", "profiles.json"
    ),
    "r",
) as f:
    fakeverse = json.load(f)

fakeverse = {profile["name"]: profile for profile in fakeverse}

list(fakeverse["Ali Rezaei"].keys())

['name',
 'nationality',
 'language',
 'occupation',
 'university',
 'degree',
 'hobby',
 'pet',
 'type_of_car',
 'allergy',
 'favorite_food',
 'favorite_drink',
 'favorite_music_genre',
 'favorite_sport',
 'favorite_boardgame',
 'favorite_color',
 'favorite_city',
 'biggest_fear']

In [55]:
from src.selection.data import SelectionSample, get_random_sample
from src.functional import generate_with_patch

# synth_entity = "Jack Wilson"
# synth_entity = "Takeshi Yamamoto"
# synth_entity = "Elara Vance"
# synth_entity = "Ayse Kaya"
synth_entity = "Rachel Levy"
# synth_entity = "James Mwangi"
# synth_entity = "Sarah MacDonald"
# synth_entity = "Jose Cruz"

prompts = [
    "{} is by profession a",
]

for prompt in prompts:
    gen = generate_with_patch(
        mt = mt,
        inputs = prompt.format(synth_entity),
        n_gen_per_prompt=1,
        remove_prefix=True,
        do_sample=False,
    )[0]
    print(f"{prompt.format(synth_entity)} >> {gen}")

sample = get_random_sample(
    people_by_category = people_by_category,
    mt = mt,
    n_distractors=5,
    get_alt_obj=True,
    category=fakeverse[synth_entity]["occupation"],
    # obj_idx=3,
    filter_by_lm_prediction=True
)
print(sample)
sample.prediction

Rachel Levy is by profession a >>  golfer, hailing from Brazil. She is a graduate of the Ludwig Maximilian University of Munich
Viktor Hovland -> Brooks Koepka (2): ['Jordan Peele', 'J.K. Rowling', 'Brooks Koepka', 'Kevin Durant', 'Will Smith', 'Ricky Gervais']


[PredictedToken(token=' Brooks', prob=0.83203125, logit=19.875, token_id=39119, metadata=None),
 PredictedToken(token=' Viktor', prob=0.046875, logit=17.0, token_id=77116, metadata=None),
 PredictedToken(token=' The', prob=0.0250244140625, logit=16.375, token_id=578, metadata=None),
 PredictedToken(token=' None', prob=0.0220947265625, logit=16.25, token_id=2290, metadata=None),
 PredictedToken(token=' B', prob=0.0118408203125, logit=15.625, token_id=426, metadata=None)]

In [57]:
#TODO (gio): fix conflicting cases; like a journalist who is also a writer

from src.functional import generate_with_patch

print("=" * 50)
sample.options[sample.obj_idx] = synth_entity

#! match with the fake entity
prompt = sample.prompt
print(prompt, ">>", synth_entity)

#! fake entity as the pivot
# prompt_template = sample.prompt.replace(sample.subj, "{}").replace(synth_entity, "{}")
# prompt = prompt_template.format(synth_entity, sample.subj)
# print(prompt, ">>", sample.subj)

generate_with_patch(
    mt = mt,
    inputs = prompt,
    n_gen_per_prompt=1,
    remove_prefix=True,
    do_sample=False,
)[0]

Which person from the following list has the profession in common with Viktor Hovland?
Options: Jordan Peele, J.K. Rowling, Rachel Levy, Kevin Durant, Will Smith, Ricky Gervais
Ans: >> Rachel Levy


' Rachel Levy\nExplanation: Viktor Hovland is a professional golfer. Rachel Levy is also a'

In [41]:
import random

#################################################
N_DISTRACTORS = 3

attribute = "occupation"
prompt_template = """Which person from the following list has the profession in common with {}?
Options: {}
Ans:"""
check_attr_prompt = "{} works as a"

# attribute = "type_of_car"
# prompt_template = """Which person from the following list drives the same car as {}?
# Options: {}
# Ans:"""
# check_attr_prompt = "{} drives a"

# attribute = "allergy"
# prompt_template = """Which person from the following list has the same allergy as {}?"
# Options: {}
# Ans:"""
# check_attr_prompt = "{} has an allergy to"

# attribute = "hobby"
# prompt_template = """Which person from the following list has the same hobby as {}?
# Options: {}
# Ans:"""
# check_attr_prompt = "{}'s hobby is"
#################################################

# pivot_entity = "Jack Wilson"
# pivot_entity = "Rajesh Kumar"
# pivot_entity = "Anna Schmidt"
# pivot_entity = "Diego Martinez"
# pivot_entity = "Rajesh Kumar"
# pivot_entity = "Mehmet Yilmaz"
# pivot_entity = "David Thompson"
# pivot_entity = "Camila Torres"
# pivot_entity = "James Mwangi"
# pivot_entity = "Ali Rezaei"
# pivot_entity = "Akosua Boateng"
pivot_entity = "Jennifer Davis"

same_attr = [
    name
    for name, profile in fakeverse.items()
    if profile[attribute] == fakeverse[pivot_entity][attribute] and name != pivot_entity
]

obj = random.choice(same_attr)
print(f"Object: {obj}")

different_attr = [
    name
    for name, profile in fakeverse.items()
    if profile[attribute] != fakeverse[pivot_entity][attribute]
]

options = random.sample(different_attr, N_DISTRACTORS)
options.append(obj)
random.shuffle(options)

# check
for subj in [pivot_entity] + options:
    gen = generate_with_patch(
        mt=mt,
        inputs= check_attr_prompt.format(subj),
        n_gen_per_prompt=1,
        remove_prefix=True,
        do_sample=False,
    )[0]
    print(f"{subj} >> {gen}")
print("=" * 50)

prompt = prompt_template.format(pivot_entity, ", ".join(options))
print(prompt)

gen = generate_with_patch(
    mt=mt,
    inputs=prompt,
    n_gen_per_prompt=1,
    remove_prefix=True,
    do_sample=False,
    max_new_tokens=50
)[0]
print(f'>> "{gen}"')

Object: Chinedu Okafor


/disk/u/arnab/miniconda3/envs/connection/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/disk/u/arnab/miniconda3/envs/connection/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:636: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


Jennifer Davis >>  professional tennis player, hailing from Israel. She is fluent in Hebrew and holds a Bachelor's degree
Chinedu Okafor >>  professional tennis player, hailing from France. He is fluent in French and has a Bachelor's degree
Maria Santos >>  fashion designer, having earned her Bachelor's degree in Fashion Design from Harvard University. Her professional career is
Anna Schmidt >>  fashion designer, having earned her Bachelor's degree in Fashion Design from Sapienza University of Rome.
Siriporn Suwannarat >>  news anchor and holds a Bachelor's degree in Political Science. Her professional career in broadcasting is complemented
Which person from the following list has the profession in common with Jennifer Davis?
Options: Chinedu Okafor, Maria Santos, Anna Schmidt, Siriporn Suwannarat
Ans:
>> " Anna Schmidt
Reason: Anna Schmidt and Jennifer Davis are both models."


In [87]:
with open("/disk/u/arnab/Codes/Projects/retrieval/results/selection/test_1_real/Llama-3.2-3B/results.json") as f:
    test_1_results = json.load(f)

In [90]:
test_1_results["accuracy"]

0.35

In [ ]:
from test_suite.test_01_real_entities import SelectionResults

sample_0 = SelectionResults.from_dict(test_1_results["samples"][0])
print(sample_0.sample.prompt, ">>", sample_0.sample.obj)

SyntaxError: invalid syntax (2957842595.py, line 4)

In [94]:
from src.functional import predict_next_token
predict_next_token(
    mt = mt,
    inputs = sample_0.sample.prompt
)

[[PredictedToken(token=' George', prob=0.90234375, logit=20.875, token_id=10058, metadata=None),
  PredictedToken(token=' The', prob=0.0308837890625, logit=17.5, token_id=578, metadata=None),
  PredictedToken(token=' C', prob=0.0308837890625, logit=17.5, token_id=356, metadata=None),
  PredictedToken(token=' None', prob=0.005706787109375, logit=15.8125, token_id=2290, metadata=None),
  PredictedToken(token=' ', prob=0.0032501220703125, logit=15.25, token_id=220, metadata=None)]]